##  <i class="fa fa-spinner"></i> 装载自定义模块
把当前repo根目录加入 Python 搜索模块的路径列表，并求得根目录

In [ ]:
import os,sys

def find_repo_root():
    # 获取当前脚本的绝对路径
    try:
        # 如果当前运行环境是 Jupyter Notebook，使用当前工作目录
        script_path = os.path.abspath('.')
    except NameError:
        # 否则，使用 __file__ 变量
        script_path = os.path.abspath(__file__)
    # 判断当前脚本目录是否存在`.git`文件夹
    if os.path.isdir(os.path.join(script_path, '.git')):
        return script_path
    # 初始化存储结果的变量
    result = None
    # 循环求当前路径的父目录，直到找到`.git`文件夹
    while True:
        # 将当前路径的父目录赋值给当前路径
        script_path = os.path.dirname(script_path)
        # 判断当前路径是否存在`.git`文件夹
        if os.path.isdir(os.path.join(script_path, '.git')):
            # 如果存在，将当前路径存储在结果变量中
            result = script_path
        # 判断当前路径是否为根目录（即是否已经搜索到最外层）
        if script_path == '/':
            # 如果是，返回结果变量的值
            return result
        
repo_root_dir=find_repo_root()
sys.path.append(repo_root_dir)
# print(repo_root_dir)
# 获取repo所在的根目录
root_dir = os.path.dirname(repo_root_dir)
# 获取当前环境名称与当前环境content路径
from func.env import detect_environment
env_cb = detect_environment()
env_name = env_cb['env_name']
content_path = env_cb['content_path']

# 装载nbfunc中的hub涉及的所有ipynb函数
%cd $repo_root_dir
%run ./nbfunc/hub.ipynb

initGit()

In [ ]:
import os

def download_clash(root_dir, clash_version, clash_install_dir, clash_run_dir):
    check_dist = os.path.join(root_dir, clash_install_dir, f'clash-linux-amd64-{clash_version}.gz')
    clash_run_dist = os.path.join(root_dir, clash_run_dir, f'clash-linux-amd64-{clash_version}.gz')
    if not os.path.exists(os.path.join(root_dir, clash_run_dir)):
        os.makedirs(os.path.join(root_dir, clash_run_dir))
    if os.path.isfile(check_dist):
        print("Clash已经下载。")
        !cp $check_dist $clash_run_dist
    else:
        !wget -P {root_dir}/{clash_install_dir} https://github.com/Dreamacro/clash/releases/download/{clash_version}/clash-linux-amd64-{clash_version}.gz
    !gunzip {clash_run_dist}
    clash_EXEC = os.path.join(root_dir, clash_run_dir, f'clash-linux-amd64-{clash_version}')
    clash_Sim_EXEC = os.path.join(root_dir, clash_run_dir, 'clash')
    !chmod +x $clash_EXEC
    !mv $clash_EXEC $clash_Sim_EXEC
    print("Clash已经安装。")
    
def download_config(root_dir, config_name, config_dir, mmdb_name, mmdb_dir, clash_install_dir, config_yaml_dist,origin_yaml_dist):
    clash_install_path = os.path.join(root_dir, clash_install_dir)
    mmdb_path = os.path.join(root_dir, mmdb_dir)
    config_path = os.path.join(root_dir, config_dir)

    # 下载Country.mmdb文件
    check_mmdb_dist = os.path.join(clash_install_path, mmdb_name)
    if not os.path.isfile(check_mmdb_dist):
        !wget -O {check_mmdb_dist} https://www.sub-speeder.com/client-download/{mmdb_name}
    country_mmdb_dist = os.path.join(mmdb_path, mmdb_name)
    if not os.path.isfile(country_mmdb_dist):
        !mkdir -p {mmdb_path}
        !cp {check_mmdb_dist} {country_mmdb_dist}
        print("Country.mmdb已经复制到配置目录。")
    else:
        print("Country.mmdb已经下载。")

    # 下载和复制config.yaml文件
    import shlex

    origin_yaml_dist = shlex.quote(origin_yaml_dist)
    config_yaml_dist = shlex.quote(config_yaml_dist)
    
    
    if not os.path.isfile(config_yaml_dist):
        !wget -O {config_yaml_dist} "https://raw.githubusercontent.com/Dreamacro/clash-dashboard/master/docker/clash-configs/{config_name}"
    if not os.path.isfile(config_yaml_dist):
        !mkdir -p {config_path}
        !cp {origin_yaml_dist} {config_yaml_dist}
        print("config.yaml已经复制到配置目录。")
    else:
        !cp {origin_yaml_dist} {config_yaml_dist}
        print("config.yaml已被覆盖。")

    print("Clash配置已经完成。")

import yaml

def modify_yaml_config(config_file_path, key_values_dict):
    with open(config_file_path, 'r') as f:
        config = yaml.load(f, Loader=yaml.SafeLoader)
    for key, value in key_values_dict.items():
        keys = key.split('.')
        sub_config = config
        for sub_key in keys[:-1]:
            sub_config = sub_config[sub_key]
        sub_config[keys[-1]] = value
    with open(config_file_path, 'w') as f:
        yaml.dump(config, f, default_flow_style=False)

def yaml_config_clean(yaml_file_path):
    # 读取配置文件
    with open(yaml_file_path, 'r') as f:
        config = yaml.safe_load(f)

    # 清除带有 RULE-SET 的规则
    if 'rules' in config:
        rules = config['rules']
        config['rules'] = [rule for rule in rules if not rule.startswith('RULE-SET')]

    # 保存配置文件
    with open(yaml_file_path, 'w') as f:
        yaml.dump(config, f)

    print('Clash配置已经清理。')
    
def run_clash(root_dir, config_dir, clash_run_dir,config_yaml_dist):
     
    clash_run_path = os.path.join(root_dir, clash_run_dir)
    clash_Sim_EXEC = os.path.join(clash_run_path, 'clash')
    cmd = f'{clash_Sim_EXEC} -d {os.path.dirname(config_yaml_dist)}'
    print(cmd)
    !$cmd

key_values_dict = {
    'external-controller': '127.0.0.1:7776',
    'port': 7777,
    'socks-port': 7778,
    'mixed-port': 7779,
    'mode':'Global'
}
    
clash_version = "v1.13.0"
config_name = "Dler Cloud.yaml"
config_dir = ".config/clash"
mmdb_name = "Country.mmdb"
mmdb_dir = ".config/clash"
clash_install_dir = "clashInstall"
clash_run_dir = "clash_run"
origin_yaml_dist = os.path.join(root_dir, clash_install_dir, config_name)
config_yaml_dist = os.path.join(root_dir, config_dir, 'config.yaml')

download_clash(root_dir, clash_version, clash_install_dir, clash_run_dir)
download_config(root_dir, config_name, config_dir, mmdb_name, mmdb_dir, clash_install_dir,config_yaml_dist,origin_yaml_dist)
modify_yaml_config(config_yaml_dist, key_values_dict)
yaml_config_clean(config_yaml_dist)
run_clash(root_dir, config_dir, clash_run_dir,config_yaml_dist)